# Armageddon: solver

## `solver` module
* Solve the system of differential equations describing meteoroid entry
* Calculate the kinetic energy lost per unit altitude
* Compute the burst altitude, burst energy and horizontal path length from the entry point

### Importing `Planet()` class

The class `Planet()` contains functions to solve the system of ordinary differential equations and analyse the outcomes. It is initialised with parameters for the given target planet.

In [5]:
# import the class within the module

from armageddon.solver import Planet